---
title: "MicroGrad from Scratch with Andrej Karpathy's tutorial"
author: "Jax Bulbrook"
date: "2023-07-25"
categories: [100-papers]
---

I want to learn how Transformers work in depth, but I need a bit of background first. I am going to do all of the tutorials on Andrej Karpathy's channel, starting with [this micrograd from scratch one](https://www.youtube.com/watch?v=VMj-3S1tku0). That should give me enough background to understand his latest GPT from Scratch tutorial!

In [ ]:
class Value:
    